# **SVM dengan K-Fold Cross Validation**

## **Library Sastrawi**

In [1]:
!pip install Sastrawi

## **Library Preprocessing**

In [1]:
import string
import re
import numpy as np
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

## **Library feature Extraction/Classifier/Evaluation**

In [2]:
from sklearn import preprocessing, model_selection, svm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

## **Libaray Visualization**

In [3]:
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [4]:
# word_tokenize & FreqDist from nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

# stopwords
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /home/pythonku/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pythonku/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
np.random.seed(123)

##**Baca Dataset**

> Sebelum Case Folding

In [ ]:
data = pd.read_csv("Dataset.csv", sep=";", encoding='latin1')
data['tweet'].str.encode('ascii','ignore')
data.head()

In [5]:
data = pd.read_csv("data_after_preprocessing.csv",sep=';', encoding='utf-8')

In [6]:
data.head()

,Unnamed: 0,Unnamed: 0.1,Value,tweet,cleaning,tweet_tokens,tweet_stopwords,normalization,text_stemming,category
0,0,0,Negatif,Yuk kenali setiap warna yang ada dari PeduliLi...,yuk kenali setiap warna yang ada dari pedulili...,"['yuk', 'kenali', 'setiap', 'warna', 'yang', '...","['kenali', 'warna', 'pedulilindungi']",kenali warna pedulilindungi,kenal warna pedulilindungi,0
1,1,1,Negatif,Yth @PLindungi @KemenkesRI status saya di apli...,yth status saya di aplikasi peduli lindungi ti...,"['yth', 'status', 'saya', 'di', 'aplikasi', 'p...","['status', 'aplikasi', 'peduli', 'lindungi', '...",status aplikasi peduli lindungi berubah hitam ...,status aplikasi peduli lindung ubah hitam test...,0
2,2,2,Negatif,"Yang terakhir, EHac di app peduli lindungi ini...",yang terakhir ehac di app peduli lindungi ini ...,"['yang', 'terakhir', 'ehac', 'di', 'app', 'ped...","['ehac', 'app', 'peduli', 'lindungi', 'jelek',...",ehac app peduli lindungi jelek app didevelop b...,ehac app peduli lindung jelek app didevelop be...,0
3,3,3,Negatif,"yaallah rima udah pede banget jalan, udah scan...",yaallah rima udah pede banget jalan udah scan ...,"['yaallah', 'rima', 'udah', 'pede', 'banget', ...","['yaallah', 'rima', 'pede', 'banget', 'jalan',...",yaallah rima percaya diri banget jalan scan pe...,yaallah rima percaya diri banget jalan scan pe...,0
4,4,4,Negatif,ya udahlah... udah bebaslah kemana-mana. kagak...,ya udahlah udah bebaslah kemanamana kagak ada ...,"['ya', 'udahlah', 'udah', 'bebaslah', 'kemanam...","['bebaslah', 'kemanamana', 'sekatsekatan', 'pc...",bebaslah kemanamana sekatsekatan pcrpcr aplika...,bebas kemanamana sekatsekatan pcrpcr aplikasi ...,0


> Sesudah Case Folding

## Text Cleaning

>Karakter Spesial

In [ ]:
def remove_tweet_special(text):
  # Remove Tab, New Line, ans Back Slice
  text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
  # Remove non ASCII (emoticon, chinese word, etc)
  text = text.encode('ascii', 'replace').decode('ascii')
  # Remove mention, link, hastag
  text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
  # Remove incomplete URL
  text = text.replace("http://", " ").replace("https://", " ")
  # Remove Number
  text = re.sub(r"\d+", "", text)
  # Remove Punctuation
  text = text.translate(str.maketrans("","",string.punctuation))
  # whitespace
  text = text.strip()
  # whitiespace multiple
  text = re.sub('\s+',' ',text)
  #single char
  text = re.sub(r"\b[a-zA-Z]\b", "", text)
    #make lower text
  text = str.lower(text)
  return text

data['cleaning'] = data['tweet'].apply(remove_tweet_special)

In [ ]:
data.head()

In [ ]:
data.to_csv("dataset_text_after_cleaning.csv",sep=';')

##**Tokenizing**

In [ ]:
# NLTK word tokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

data['tweet_tokens'] = data['cleaning'].apply(word_tokenize_wrapper)

In [ ]:
print('Tokenizing Result : \n') 
print(data['tweet_tokens'].head())

NLTK calc Frequently Distribution

In [ ]:
data.to_csv("dataset_text_after_tokenizing.csv",sep=';')

In [ ]:
def freqDist_wrapper(text):
  return FreqDist(text)

data['tweet_tokens_fdist'] = data['tweet_tokens'].apply(freqDist_wrapper)

In [ ]:
print('Frequency Tokens : \n') 
print(data['tweet_tokens_fdist'].head().apply(lambda x : x.most_common()))

## Stopwords

>Stopwords Indonesia

In [ ]:
list_stopwords = stopwords.words('indonesian')
#print(list_stopwords)

>Stopwords Additional

In [ ]:
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])
#print(list_stopwords)

>Stopwords dari File

In [ ]:
txt_stopwords = pd.read_csv("stopwords.txt",names=["stopwords"], header=None)
list_stopwords.extend(txt_stopwords["stopwords"][0].split(' '))

In [ ]:
print(list_stopwords)

>Convert List to Dictionary

In [ ]:
list_stopwords = set(list_stopwords)

> Menghapus Stopwords pada list token

In [ ]:
def stopwords_removal(words):
  return [word for word in words if word not in list_stopwords]

data['tweet_stopwords'] = data['tweet_tokens'].apply(stopwords_removal)

In [ ]:
print(data['tweet_stopwords'].head(5))

In [ ]:
data.to_csv('data_text_after_stopwords.csv',sep=';')

##Normalization

In [ ]:
normalization_word = pd.read_csv("colloquial-indonesian-lexicon.csv")
normalization_word_dict = {}

for index, row in normalization_word.iterrows():
  if row[0] not in normalization_word_dict:
    normalization_word_dict[row[0]] = row[1]

def normalization(document):
  return [normalization_word_dict[term] if term in normalization_word_dict else term for term in document]

data['normalization'] = data['tweet_stopwords'].apply(normalization)


In [ ]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
data['normalization']=data['normalization'].apply(lambda x: TreebankWordDetokenizer().detokenize(x))
data.head()

In [ ]:
data.to_csv('data_text_after_normalization.csv',sep=';')

## Stemming

In [ ]:
def stemming(text):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  stem = stemmer.stem(text)
  return stem

data['text_stemming'] = data['normalization'].apply(lambda text: stemming(text))

In [ ]:
data.head()

In [ ]:
data.to_csv("data_text_after_stemming.csv", sep=';')

In [ ]:
data.to_csv("data_after_preprocessing.csv", sep=';',encoding='utf-8')

##Memberi Label

In [7]:
def labelToNumeric(category):
  if category == 'Negatif':
    return 0
  elif category == 'Positif':
    return 1
  else :
    return 0
data['category'] = data['Value'].apply(labelToNumeric)

In [8]:
positif,negatif=data['category'].value_counts()

In [ ]:
data['category']

In [ ]:
tfid_vectorized = TfidfVectorizer()
tfid_vectorized.fit(data['tweet'])
dictionary = tfid_vectorized.vocabulary_.items()
vocab = []
count = []
for key, value in dictionary:
  vocab.append(key)
  count.append(value)
vocab_bef_stem = pd.Series(count, index=vocab)
vocab_bef_stem = vocab_bef_stem.sort_values(ascending=True)
top_vocab = vocab_bef_stem.head(30)
top_vocab.plot(kind = 'barh', figsize=(5,10))

In [8]:
X_train, X_test, y_train, y_test = model_selection.train_test_split( data['text_stemming'].values.astype('U'), data['category'], test_size=0.2, random_state=45)

## Feature Extraction/Pembobotan TF-IDF

>Bag of Words Model

In [9]:
cv = CountVectorizer()
X = cv.fit_transform(data['text_stemming'].values.astype('U')).toarray()
Y = data['category']

In [ ]:
import pickle
pickle.dump(cv,open('cv.model'))

>Creating TF-IDF

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

In [13]:
print(X)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## CRoss Validation

In [15]:
from sklearn.model_selection import cross_val_score
clf = svm.SVC(C=0.5, degree=2, gamma='scale', kernel='sigmoid')
scores = cross_val_score(clf,X,Y, cv=10)

In [16]:
print(scores)
scores.mean()

[0.73259053 0.77437326 0.86629526 0.83844011 0.75766017 0.69916435
 0.66295265 0.82729805 0.8547486  0.72905028]


0.7742573255940618

In [17]:
from sklearn.naive_bayes import MultinomialNB
clf2 = MultinomialNB(fit_prior=False)
scores2 = cross_val_score(clf2,X,Y, cv=10)

In [18]:
print(scores2)
scores2.mean()

[0.75487465 0.80501393 0.85236769 0.83286908 0.76044568 0.69359331
 0.70473538 0.82729805 0.82122905 0.75977654]


0.7812203358179921

## Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm
svm = svm.SVC()
param = {'C':(0.5,1),
         'kernel':('rbf','linear','sigmoid','poly'),
         'gamma':('scale','auto'),
         'degree':(2,3,4)
         }
grid = GridSearchCV(svm, param)
grid.fit(X,Y)
grid.best_params_

In [ ]:
grid.best_score_

## Split Dataset

>Splitting Dataset into training and testing

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=45)

In [ ]:
print(X)

## SVM

In [20]:
from sklearn import svm
import time

classifierSVM = svm.SVC(C=0.5,degree=2, gamma='scale', kernel='sigmoid')

>Proses Pelatihan

In [21]:
t0 = time.time()
classifierSVM.fit(X_train, y_train)
t1 = time.time()

>Prediksi Dataset

In [23]:
y_pred_SVM = classifierSVM.predict(X_test)
t2 = time.time()
time_train=t1-t0
time_predict=t2-t1

In [24]:
#time
print("Training Time: %fs, Prediction Time: %fs"%(time_train, time_predict))

Training Time: 20.120252s, Prediction Time: 84.065832s


In [26]:
import pickle

In [27]:
pickle.dump(classifierSVM,open('classifierSVM.model','wb'))

In [28]:
from sklearn.metrics import confusion_matrix
cm_SVM = confusion_matrix(y_test, y_pred_SVM)
print('confusin matrix :')
print(cm_SVM)

from sklearn.metrics import accuracy_score
acc = (accuracy_score(y_test, y_pred_SVM))
print("Accuracy : ", (acc))

from sklearn.metrics import precision_score
pcc = (precision_score(y_test, y_pred_SVM))
print("Preccision : ",(pcc))

from sklearn.metrics import recall_score
rec = (recall_score(y_test, y_pred_SVM))
print("Recall : ", (rec))

from sklearn.metrics import f1_score
f1= (f1_score(y_test, y_pred_SVM))
print("Skor F1: ", (f1))

confusin matrix :
[[278  72]
 [ 73 295]]
Accuracy :  0.7980501392757661
Preccision :  0.8038147138964578
Recall :  0.8016304347826086
Skor F1:  0.8027210884353742


In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_SVM))

              precision    recall  f1-score   support

           0       0.79      0.79      0.79       350
           1       0.80      0.80      0.80       368

    accuracy                           0.80       718
   macro avg       0.80      0.80      0.80       718
weighted avg       0.80      0.80      0.80       718



## Prediksi SVM

In [ ]:
def prediksi_SVM(text):
  text = remove_tweet_special(text)
  text = str.lower(text)
  text = word_tokenize_wrapper(text)
  text = stopwords_removal(text)
  text = normalization(text)
  text = TreebankWordDetokenizer().detokenize(text)
  text = stemming(text)
  text = cv.transform([text]).toarray()
  text = tfidfconverter.transform(text).toarray()
  text = classifierSVM.predict(text)
  if text[0] == 0:
    print('Sentimen Negatif')
  else:
    print('Sentimen Positif')
  return text

In [ ]:
SVM = prediksi_SVM("Indonesia Vaksin gratis")

## Naive Bayes

In [30]:
from sklearn.naive_bayes import MultinomialNB
import time

classifierNB = MultinomialNB(fit_prior=False)
#proses pelatihan
t0 = time.time()
classifierNB.fit(X_train, y_train)
t1= time.time()
#prediksi 
y_pred_NB= classifierNB.predict(X_test)
t2=time.time()
time_train=t1-t0
time_predict=t2-t1

In [31]:
#time
print("Training Time: %fs, Prediction Time: %fs"%(time_train, time_predict))

Training Time: 0.055861s, Prediction Time: 0.009877s


In [33]:
pickle.dump(classifierNB,open('classifierNB.model','wb'))

In [ ]:
from sklearn.metrics import confusion_matrix
cm_NB = confusion_matrix(y_test, y_pred_NB)
print("confusion matrix :")
print(cm_NB)

from sklearn.metrics import accuracy_score
acc = (accuracy_score(y_test, y_pred_NB))
print("Accuracy : ", (acc))

from sklearn.metrics import precision_score
pre = (precision_score(y_test, y_pred_NB))
print("Precision : ", (pre))

from sklearn.metrics import recall_score
rec= (recall_score(y_test, y_pred_NB))
print("Recall : ", (rec))

from sklearn.metrics import f1_score
f1= (f1_score(y_test, y_pred_NB))
print("Skor F1: ", (f1))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_NB))

##Prediksi Naive Bayes

In [ ]:
def prediksi_NB(text):
  text = remove_tweet_special(text)
  text = str.lower(text)
  text = word_tokenize_wrapper(text)
  text = stopwords_removal(text)
  text = normalization(text)
  text = TreebankWordDetokenizer().detokenize(text)
  text = stemming(text)
  text = cv.transform([text]).toarray()
  text = tfidfconverter.transform(text).toarray()
  text = classifierNB.predict(text)
  if text[0] == 0:
    print('Sentimen Negatif')
  else:
    print('Sentimen Positif')
  return text

In [ ]:
Naive_Bayes = prediksi_NB("Indonesia Vaksin gratis")

In [ ]:
print(type(classifierNB))